# Tutorial 4: Q-learning with Tabular Methods

In this tutorial we will finally see how to implement a Q-learning algorithm with Tabular Methods for our game.

We gathered all the functions written in the previous tutorial in a script encode.py, that you can find in the Modules folder. In this way we can call the functions from there without having to redefine them. To do that we insert that folder among the paths that python searches for modules and libraries and then import the module as "cod"; you can call any function from that module as cod.function.

## Outline
1. define **greedy** and **$\epsilon$-greedy** policies, to choose actions given the state;
2. define an **update rule** for the Q-values, that is the way our algorithm is going to learn;
3. define a minimal **training cycle** for our agent.

In [1]:
import sys
sys.path.insert(0, "../Environment/")
sys.path.insert(0, "../Modules/")
import halite_env as Env
import encode as cod

In [2]:
import numpy as np
import matplotlib.pyplot as plt

## Policies
This two policies are very simple and intuitive policies. 

The greedy one simply says: "given the state in which I am, I want to choose what I think is the best action for me", that is given $s$, choose $argmax_a Q(s,a)$. 

The epsilon-greedy policy is similar, but with probability epsilon (that as the name might suggest, is usually small) choose an action at random.

In [3]:
def greedy_policy(s_enc, q_values):
    """
    Parameters
    ----------
    s_enc : int, encoded state
    q_values : numpy matrix containing the Q-values
    
    Returns
    -------
    a : action that maximizes the Q-value given s_enc
    """
    return np.argmax(q_values[s_enc])

def e_greedy_policy(s_enc, q_values, eps = 0.01):
    """
    Parameters
    ----------
    s_enc : int, encoded state
    q_values : numpy matrix containing the Q-values
    eps : float representing the probability of choosing exploration over exploitation
    
    Returns
    -------
    a : random action with prob. eps and action that maximizes the Q-value given s_enc 
        with prob. (1-eps) 
    """
    u = np.random.rand()
    if u > eps:
        return np.argmax(q_values[s_enc])
    else:
        return np.random.randint(0, len(q_values[s_enc]))


The only possible problem with this implementation is that argmax is forced by numpy to return the first index if there are more global maxima. 

In [4]:
#example
M = np.full(shape=(3,3),fill_value=0)
M[1]
greedy_policy(1, M)

0

## Update rule

As we discussed in the second tutorial, the update rule is

$$Q(s,a) = (1-\alpha)Q(s,a) + \alpha [r(s,a) + max_{a'}Q(s',a')]$$
where 
- $\alpha$ is called learning rate and controls the trade-off between learning speed and algorithmic stability;
- $s$ is the previous state;
- $s'$ is the current state;
- $a$ is the action that has taken us from $s$ to $s'$ (may be in a stochastic way);
- $r(s,a)$ is the reward received this turn for the previous step
- $a'$ is the action that we would take following the greedy policy in state $s'$ **BEFORE** the update.

Implementing this simple rule we get the following function. We also introduced the possibility to use a discount factor gamma, that encodes the fact that "it's better to get one euro today than tomorrow". Anyway for our application is not relevant and we can keep it fixed at 1.

In [5]:
def update_q_v0(s_enc, a, r, sp_enc, ap, q_values, alpha = 0.1, gamma = 1):
    """
    Update rule of Q-learning.
    
    Parameters
    ----------
    s_enc: previous encoded state
    a : action taken last turn
    r : reward obtained for last action
    sp_enc: current encoded state
    ap : action that the agent would take following the greedy policy in state 𝑠p BEFORE the update
    q_values : numpy matrix containing the Q-values
    alpha : learning rate
    gamma : discount rate
    
    Returns
    -------
    updated Q-values
    
    """
    q_values[s_enc,a] = (1-alpha)*q_values[s_enc,a] + alpha*(r + gamma*q_values[sp_enc,ap])
    return q_values

A problem that we encountered with this update rule is that it is thought for a single episodic task (like "go out in the sea, get some fish and come back!") but our ship has to do it many times in a single match and this messes things up. What we want to do is to make the ship belive that it has finished its work once it comes back to the shipyard; therfore the term $max_{a'}Q(s',a')$ is set to zero if $s'$ is a terminal state (i.e. the ship is in the shipyard).

We are not 100% sure of why this works and why the previous update not, but euristically there is a huge difference, namely in the previous case the ship learns not to pass from the shipyard, whereas in the new case everything works more or less fine.

Anyway, here's the new update rule:

In [6]:
def update_q_v1(s, a, r, sp, ap, q_values, alpha = 0.1, gamma = 1, map_size = 7, h_lev = 3, n_actions = 5):
    """
    Update rule of Q-learning. Enforces Q-value = 0 for the terminal state.
    
    Parameters
    ----------
    s_enc: previous encoded state
    a : action taken last turn
    r : reward obtained for last action
    sp_enc: current encoded state
    ap : action that the agent would take following the greedy policy in state 𝑠p BEFORE the update
    q_values : numpy matrix containing the Q-values
    alpha : learning rate
    gamma : discount rate
    map_size : linear dimension of the squared map
    h_lev : number of quantization levels for the halite
    n_actions : number of actions that the agent can choose
    
    Returns
    -------
    updated Q-values
    
    """
    n_cells = map_size**2
    s_dec = cod.decode3D(s_enc, L1 = n_cells, L2 = h_lev**6, L3 = n_actions-1)
    sp_dec = cod.decode3D(sp_enc, L1 = n_cells, L2 = h_lev**6, L3 = n_actions-1)
    shipy_pos = (n_cells-1)/2 #shipyard is at the center of the map
    
    if (sp_dec[0] == shipy_pos and s_dec[0] != shipy_pos):
        # sp is terminal state -> enforce to have Q-value = 0 for all actions ap
        q_values[s_enc,a] = (1-alpha)*q_values[s_enc,a] + alpha*r 
    else:
        q_values[s_enc,a] = (1-alpha)*q_values[s_enc,a] + alpha*(r + gamma*q_values[sp_enc,ap]) # normal update
    return q_values

## Training cycle

In [14]:
#@@@@@@@@@@@@@@@@@@@@@@
# Environment variables
#@@@@@@@@@@@@@@@@@@@@@@
NUM_PLAYERS = 1
MAP_SIZE = 7 # 7 x 7 map
TOT_TURNS = 400 # number of turns for each episode

#@@@@@@@@@@@@@@@@
# State variables
#@@@@@@@@@@@@@@@@
H_LEV = 3 # halite levels
N_CELLS = MAP_SIZE**2 # number of cells in a square map
N_STATES = N_CELLS*H_LEV**6*4
N_ACTIONS = 5 # no dropoffs, 1 action for staying still, 4 for moving in the cardinal directions
print("Total number of states to be experienced: ", N_STATES)

#@@@@@@@@@@@@@@@@@@@@
# Learning parameters
#@@@@@@@@@@@@@@@@@@@@
N_BATCH = 10 #100 # number of episodes in an epoch
MAX_EPOCHS = 200 # max number of epochs played before stopping (500 ~ 7.3 hours of training)
DISCOUNT_FACTOR = 1 - 1/TOT_TURNS #train ships as if each turn has a probability of 1/tot_turns of ending the game 
STD_REWARD = -0.01
LEARNING_RATE = 0.1
EPS_START = 0.5

Total number of states to be experienced:  142884


In [8]:
#@@@@@@@@@@@@@@@@@@@
# Learning variables
#@@@@@@@@@@@@@@@@@@@
q_values = np.zeros((N_STATES,N_ACTIONS)) #initialize to zero
#q_values = np.load("Q_values.npy") # or re-use the one already trained

So how it works an episode?

In [10]:
env = Env.HaliteEnv(NUM_PLAYERS, MAP_SIZE, episode_lenght = TOT_TURNS) # init environment
steps = 0
reward = 0 # cumulative reward of the episode
eps = EPS_START
# first mandatory step
steps = steps + 1
print("\nStep number %d:"%steps)
action_matrix = np.full((MAP_SIZE,MAP_SIZE), -1) # no ship, no action
shipyard_action = True # initially always choose to create a ship
# returns the matricial state, the array of players halite and a flag that is true if it's the final turn
state, players_halite, finish, _ = env.step(action_matrix, makeship = shipyard_action) 
#print("Cargo layer: \n", state[:,:,2])
current_halite = players_halite[0][0]
s_enc = cod.encode_state(state, map_size = MAP_SIZE, h_lev = H_LEV, n_actions = N_ACTIONS, debug=False)

while True:
    steps = steps + 1
    print("\nStep number %d:"%steps)
    print("Current halite: ", current_halite)
    a_enc = e_greedy_policy(s_enc, q_values, eps = eps) 
    a_mat = cod.scalar_to_matrix_action(a_enc, state, map_size = MAP_SIZE) #convert the action in matricial form

    # submit the action and get the new state
    state, players_halite, finish, _ = env.step(a_mat, makeship = False) 
    
    new_halite = players_halite[0][0]

    # compute the 1-ship reward as the halite increment of the player divided by the max halite 
    # plus a standard negative reward 
    r = (new_halite - current_halite)/1000 + STD_REWARD

    sp_enc = cod.encode_state(state, map_size = MAP_SIZE, h_lev = H_LEV, n_actions = N_ACTIONS, debug=False)
    reward += r # cumulative reward of the episode

    a_temp_enc = greedy_policy(sp_enc, q_values) # simulate the best action in the new state (before update)

    # update Q-values
    q_values = update_q_v1(s_enc, a_enc, r, sp_enc, a_temp_enc, q_values, alpha = LEARNING_RATE,
                gamma = DISCOUNT_FACTOR, map_size = MAP_SIZE, h_lev = H_LEV, n_actions = N_ACTIONS)
    
    # update states and halite
    s_enc = sp_enc
    current_halite = new_halite

    if (finish == True) or (steps >= 400):
        print("\nEnd episode.")
        break


Step number 1:

Step number 2:
Current halite:  4000.0

Step number 3:
Current halite:  4000.0

Step number 4:
Current halite:  4000.0

Step number 5:
Current halite:  4000.0

Step number 6:
Current halite:  4000.0

Step number 7:
Current halite:  4000.0

Step number 8:
Current halite:  4000.0

Step number 9:
Current halite:  4000.0

Step number 10:
Current halite:  4000.0

Step number 11:
Current halite:  4000.0

Step number 12:
Current halite:  4000.0

Step number 13:
Current halite:  4000.0

Step number 14:
Current halite:  4000.0

Step number 15:
Current halite:  4000.0

Step number 16:
Current halite:  4000.0

Step number 17:
Current halite:  4000.0

Step number 18:
Current halite:  4000.0

Step number 19:
Current halite:  4000.0

Step number 20:
Current halite:  4000.0

Step number 21:
Current halite:  4000.0

Step number 22:
Current halite:  4000.0

Step number 23:
Current halite:  4000.0

Step number 24:
Current halite:  4000.0

Step number 25:
Current halite:  4000.0

Step nu


Step number 218:
Current halite:  6240.0

Step number 219:
Current halite:  6373.0

Step number 220:
Current halite:  6373.0

Step number 221:
Current halite:  6373.0

Step number 222:
Current halite:  6373.0

Step number 223:
Current halite:  6373.0

Step number 224:
Current halite:  6373.0

Step number 225:
Current halite:  6373.0

Step number 226:
Current halite:  6373.0

Step number 227:
Current halite:  6373.0

Step number 228:
Current halite:  6373.0

Step number 229:
Current halite:  6373.0

Step number 230:
Current halite:  6373.0

Step number 231:
Current halite:  6373.0

Step number 232:
Current halite:  6373.0

Step number 233:
Current halite:  6373.0

Step number 234:
Current halite:  6373.0

Step number 235:
Current halite:  6373.0

Step number 236:
Current halite:  6373.0

Step number 237:
Current halite:  6373.0

Step number 238:
Current halite:  6373.0

Step number 239:
Current halite:  6373.0

Step number 240:
Current halite:  6373.0

Step number 241:
Current halite: 

Once we have seen that this works, we would like once again to make a function out of it, so that we can use it more concisely. The only outputs that we need are the metrics of the episode and the updated Q-values. Since the inputs taken are a lot, it can be useful (it's just a matter of style) to define a dictionary with all the environment variables used and another one with the learning parameters maybe and then pass those dictionaries to our function.

In [11]:
env_dict = dict(NUM_PLAYERS = 1, TOT_TURNS = 400)
env_dict

{'NUM_PLAYERS': 1, 'TOT_TURNS': 400}

In [12]:
state_dict = dict(MAP_SIZE = 7,  H_LEV = 3, N_ACTIONS = 5)
state_dict

{'MAP_SIZE': 7, 'H_LEV': 3, 'N_ACTIONS': 5}

In [15]:
learning_dict = dict(LEARNING_RATE = 0.1, DISCOUNT_FACTOR = DISCOUNT_FACTOR , eps = EPS_START, STD_REWARD = STD_REWARD)
learning_dict

{'LEARNING_RATE': 0.1,
 'DISCOUNT_FACTOR': 0.9975,
 'eps': 0.5,
 'STD_REWARD': -0.01}

In [17]:
def play_episode(q_values, eps, NUM_PLAYERS, MAP_SIZE, TOT_TURNS, N_ACTIONS, H_LEV,
                 STD_REWARD,LEARNING_RATE, DISCOUNT_FACTOR, verbose = False):
    
    env = Env.HaliteEnv(NUM_PLAYERS, MAP_SIZE, episode_lenght = TOT_TURNS) # init environment
    steps = 0
    reward = 0 # cumulative reward of the episode

    # first mandatory step
    steps = steps + 1
    if verbose:
        print("\nStep number %d:"%steps)
    action_matrix = np.full((MAP_SIZE,MAP_SIZE), -1) # no ship, no action
    shipyard_action = True # initially always choose to create a ship
    # returns the matricial state, the array of players halite and a flag that is true if it's the final turn
    state, players_halite, finish, _ = env.step(action_matrix, makeship = shipyard_action) 
    #print("Cargo layer: \n", state[:,:,2])
    current_halite = players_halite[0][0]
    s_enc = cod.encode_state(state, map_size = MAP_SIZE, h_lev = H_LEV, n_actions = N_ACTIONS, debug=False)

    while True:
        steps = steps + 1
        if verbose:
            print("\nStep number %d:"%steps)
            print("Current halite: ", current_halite)
        a_enc = e_greedy_policy(s_enc, q_values, eps = eps)
        a_mat = cod.scalar_to_matrix_action(a_enc, state, map_size = MAP_SIZE) #convert the action in matricial form

        # submit the action and get the new state
        state, players_halite, finish, _ = env.step(a_mat, makeship = False) 

        new_halite = players_halite[0][0]

        # compute the 1-ship reward as the halite increment of the player divided by the max halite 
        # plus a standard negative reward 
        r = (new_halite - current_halite)/1000 + STD_REWARD

        sp_enc = cod.encode_state(state, map_size = MAP_SIZE, h_lev = H_LEV, n_actions = N_ACTIONS, debug=False)
        reward += r # cumulative reward of the episode

        a_temp_enc = greedy_policy(sp_enc, q_values) # simulate the best action in the new state (before update)

        # update Q-values
        q_values = update_q_v1(s_enc, a_enc, r, sp_enc, a_temp_enc, q_values, alpha = LEARNING_RATE,
                    gamma = DISCOUNT_FACTOR, map_size = MAP_SIZE, h_lev = H_LEV, n_actions = N_ACTIONS)

        # update states and halite
        s_enc = sp_enc
        current_halite = new_halite

        if (finish == True) or (steps >= 400):
            if verbose:
                print("\nEnd episode.")
            break
    return q_values, reward

In [18]:
q_values, reward = play_episode(q_values, verbose = True, **env_dict, **state_dict, **learning_dict)


Step number 1:

Step number 2:
Current halite:  4000.0

Step number 3:
Current halite:  4000.0

Step number 4:
Current halite:  4000.0

Step number 5:
Current halite:  4000.0

Step number 6:
Current halite:  4000.0

Step number 7:
Current halite:  4000.0

Step number 8:
Current halite:  4000.0

Step number 9:
Current halite:  4000.0

Step number 10:
Current halite:  4000.0

Step number 11:
Current halite:  4000.0

Step number 12:
Current halite:  4000.0

Step number 13:
Current halite:  4000.0

Step number 14:
Current halite:  4000.0

Step number 15:
Current halite:  4143.0

Step number 16:
Current halite:  4143.0

Step number 17:
Current halite:  4143.0

Step number 18:
Current halite:  4143.0

Step number 19:
Current halite:  4143.0

Step number 20:
Current halite:  4143.0

Step number 21:
Current halite:  4143.0

Step number 22:
Current halite:  4143.0

Step number 23:
Current halite:  4143.0

Step number 24:
Current halite:  4143.0

Step number 25:
Current halite:  4143.0

Step nu


Step number 200:
Current halite:  4143.0

Step number 201:
Current halite:  4143.0

Step number 202:
Current halite:  4143.0

Step number 203:
Current halite:  4143.0

Step number 204:
Current halite:  4143.0

Step number 205:
Current halite:  4143.0

Step number 206:
Current halite:  4143.0

Step number 207:
Current halite:  4143.0

Step number 208:
Current halite:  4143.0

Step number 209:
Current halite:  4143.0

Step number 210:
Current halite:  4143.0

Step number 211:
Current halite:  4143.0

Step number 212:
Current halite:  4143.0

Step number 213:
Current halite:  4143.0

Step number 214:
Current halite:  4143.0

Step number 215:
Current halite:  4143.0

Step number 216:
Current halite:  5143.0

Step number 217:
Current halite:  5143.0

Step number 218:
Current halite:  5143.0

Step number 219:
Current halite:  5143.0

Step number 220:
Current halite:  5143.0

Step number 221:
Current halite:  5143.0

Step number 222:
Current halite:  5143.0

Step number 223:
Current halite: 

In [19]:
halite_coll = (reward-STD_REWARD*(TOT_TURNS-1))*1000
halite_coll 

2916.9999999999977

It will be useful also to define a variable to store the main metric in learning, that is the reward

In [20]:
reward_score = np.zeros(MAX_EPOCHS) 
epochs = 0

Also we warmly suggest to use tnrange instead of range to monitor the progress of the learning cycle from the point of view of time. This is already provided if you are using our docker or if you followed the instructions to set up all the requirements in the anaconda environment.

In [21]:
#! pip install tqdm
from tqdm import tnrange

In [23]:
eps = EPS_START # starting value of epsilon
# generate an adaptive epsilon greedy algorithm, calibrated in order to have epsilon = 10^-4 at the last epoch
epsilons = np.array(list(map(lambda i : eps*np.exp(-i*2*np.log(10)/MAX_EPOCHS), np.arange(0,MAX_EPOCHS+1))))

for k in tnrange(MAX_EPOCHS):
    # here starts an epoch
    epochs = epochs + 1
    reward_progress = np.zeros(N_BATCH) # bunch of N_BATCH episodes
    eps = epsilons[epochs]
    # update the dictionary at each epoch with the new epsilon
    learning_dict = dict(LEARNING_RATE = 0.1, DISCOUNT_FACTOR = DISCOUNT_FACTOR , eps = eps, STD_REWARD = STD_REWARD)
    
    for i in range(N_BATCH):
        # here starts an episode
        q_values, reward = play_episode(q_values, **env_dict, **state_dict, **learning_dict)
        reward_progress[i] = reward
         
        #break # play just 1 episode
                
    #break # play just 1 epoch
    
    print("Average reward per episode in epoch %d: %.3f"%(epochs, reward_progress.mean()))
    reward_score[epochs-1] = reward_progress.mean()
    
    if epochs >= MAX_EPOCHS:
        print("Hey, I think you've had enough! Let's stop here.")
        break

Average reward per episode in epoch 1: -1.641
Average reward per episode in epoch 2: -1.599


KeyboardInterrupt: 

In [38]:
# finally let's save our learned Q-values
np.save("Q_values/Q_values", q_valuesvaluesvaluesvaluesvalues)

*Congratulations!* You have trained your first halite agent! As you may have experienced if you have tried to train untill the end the agent, to wait an hour or more reading just numbers is excruciating. In the next tutorial we will see a couple of tricks to better visualize how the training is going. Also we will try to visualize after the training what can we infer from the Q-values.